<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
launch_sites = {
    "CCAFS LC-40": (28.562302, -80.577356),
    "CCAFS SLC-40": (28.563197, -80.576820),
    "KSC LC-39A":  (28.608390, -80.604333),
    "VAFB SLC-4E": (34.632834, -120.610745)
}


# Start the map centered near Florida launch sites
spacex_map = folium.Map(location=[28.5, -80.6], zoom_start=6)

# Add markers for each launch site
for site, (lat, lon) in launch_sites.items():
    folium.Marker(
        location=[lat, lon],
        popup=site,
        icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
    ).add_to(spacex_map)

spacex_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circle + popup for each launch site
for _, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    
    # Add a circle around the site
    folium.Circle(
        location=coord,
        radius=10000,          # 10 km radius (adjust if needed)
        color='blue',
        fill=True,
        fill_opacity=0.2
    ).add_to(site_map)

    # Add a marker with popup
    folium.Marker(
        location=coord,
        popup=row['Launch Site'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(site_map)

site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
# Task 2: Mark the success/failed launches for each site on the map
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch record
for _, row in spacex_df.iterrows():
    coord = [row['Lat'], row['Long']]
    
    # Choose marker color based on outcome
    if row['class'] == 1:
        color = 'green'     # success
    else:
        color = 'red'       # failure
    
    # Add the marker
    folium.Marker(
        location=coord,
        popup=f"{row['Launch Site']} — {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=color)
    ).add_to(site_map)

site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:

# Apply a function to check the value of `class` column
# Function to set marker color based on launch outcome
def get_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Initialize map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for _, row in spacex_df.iterrows():
    
    marker_color = get_marker_color(row['class'])   # <— use your function
    
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} — {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    ).add_to(site_map)

site_map


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
def get_marker_color(outcome):
    return 'green' if outcome == 1 else 'red'

# Create a new column in the dataframe
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)


marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"{record['Launch Site']} — {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(
            color='white',
            icon_color=record['marker_color'],  # now the column exists
            icon='rocket',
            prefix='fa'
        )
    )
    
    marker_cluster.add_child(marker)

site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [15]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Suppose you already know the nearest city to a launch site:
from math import radians, sin, cos, sqrt, atan2
def calculate_distance(lat1, lon1, lat2, lon2):
    # Radius of Earth in km
    R = 6371.0

    # Convert coords to radians
    lat1_r = radians(lat1)
    lon1_r = radians(lon1)
    lat2_r = radians(lat2)
    lon2_r = radians(lon2)

    # Haversine formula
    dlat = lat2_r - lat1_r
    dlon = lon2_r - lon1_r

    a = sin(dlat/2)**2 + cos(lat1_r) * cos(lat2_r) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
launch_lat = launch_sites_df.loc[0, 'Lat']
launch_lon = launch_sites_df.loc[0, 'Long']

city_lat = 28.101
city_lon = -80.645

distance_km = calculate_distance(launch_lat, launch_lon, city_lat, city_lon)
distance_km
# Add a line on the map between launch site and city
folium.PolyLine(
    locations=[
        [launch_lat, launch_lon],
        [city_lat, city_lon]
    ],
    color='blue'
).add_to(site_map)

# Add distance popup to the city marker
folium.Marker(
    location=[city_lat, city_lon],
    popup=f"City — Distance: {distance_km:.2f} km"
).add_to(site_map)



Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [21]:
# Add Mouse Position tool
from folium.plugins import MousePosition
mouse_position = MousePosition(position='topright', separator=' | ', lng_first=True, num_digits=6)
mouse_position.add_to(site_map)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [25]:
import folium
import math
from folium.plugins import MousePosition

# -------------------------------------
# 2. SELECT THE COASTLINE POINT
# Use MousePosition to read the coords
# Replace these with the coordinates you selected
# -------------------------------------
coast_lat = 28.56342     # <-- change these
coast_lon = -80.56711    # <-- change these

coast_point = (coast_lat, coast_lon)

folium.Marker(
    location=[coast_lat, coast_lon],
    popup="Closest Coastline Point",
    icon=folium.Icon(color="green")
).add_to(site_map)

# -------------------------------------
# 3. SELECT THE LAUNCH SITE
# Example: CCAFS SLC-40
# Replace with your launch site values if needed
# -------------------------------------
launch_site = (28.561857, -80.577366)

folium.Marker(
    location=launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="red")
).add_to(site_map)

# -------------------------------------
# 4. Haversine Distance Function
# -------------------------------------
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Earth radius in kilometers
    R = 6371
    return R * c

# -------------------------------------
# 5. Compute the distance
# -------------------------------------
distance_km = haversine(launch_site, coast_point)
print("Distance from launch site to coastline:", distance_km, "km")

# -------------------------------------
# 6. Draw a line on the map
# -------------------------------------
folium.PolyLine(
    [launch_site, coast_point],
    color="red",
    weight=2
).add_to(site_map)

# Display the map
site_map


Distance from launch site to coastline: 1.0165874222719928 km


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [26]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)


<class 'NameError'>: name 'lines' is not defined

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [32]:
city_lat = 28.56342     # <-- change these
city_lon = -80.56711    # <-- change these

rail_lat = 28.52962     # <-- change these
rail_lon = -80.791402    # <-- change these

hwy_lat = 28.573725     # <-- change these
hwy_lon = -80.655463    # <-- change these

# -------------------------------------
# 2. SELECT THE COASTLINE POINT
# Use MousePosition to read the coords
# Replace these with the coordinates you selected
# -------------------------------------

city_point = (city_lat, city_lon)
rail_point = (rail_lat, rail_lon)
hwy_point = (hwy_lat, hwy_lon)

folium.Marker(
    location=[city_lat, city_lon],
    popup="Closest City Point",
    icon=folium.Icon(color="green")
).add_to(site_map)

folium.Marker(
    location=[rail_lat, rail_lon],
    popup="Closest Rail Point",
    icon=folium.Icon(color="green")
).add_to(site_map)

folium.Marker(
    location=[hwy_lat, hwy_lon],
    popup="Closest Highway Point",
    icon=folium.Icon(color="green")
).add_to(site_map)

# -------------------------------------
# 3. SELECT THE LAUNCH SITE
# Example: CCAFS SLC-40
# Replace with your launch site values if needed
# -------------------------------------
launch_site = (28.561857, -80.577366)

folium.Marker(
    location=launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="red")
).add_to(site_map)

# -------------------------------------
# 4. Haversine Distance Function
# -------------------------------------
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Earth radius in kilometers
    R = 6371
    return R * c

# -------------------------------------
# 5. Compute the distance
# -------------------------------------
distance_km = haversine(launch_site, city_point)
print("Distance from launch site to city:", distance_km, "km")

distance_km = haversine(launch_site, rail_point)
print("Distance from launch site to railway:", distance_km, "km")

distance_km = haversine(launch_site, hwy_point)
print("Distance from launch site to highway:", distance_km, "km")

# -------------------------------------
# 6. Draw a line on the map
# -------------------------------------
folium.PolyLine(
    [launch_site, city_point],
    color="red",
    weight=2
).add_to(site_map)

folium.PolyLine(
    [launch_site, rail_point],
    color="red",
    weight=2
).add_to(site_map)

folium.PolyLine(
    [launch_site, hwy_point],
    color="red",
    weight=2
).add_to(site_map)

# Display the map
site_map

Distance from launch site to city: 1.0165874222719928 km
Distance from launch site to railway: 21.21160083930671 km
Distance from launch site to highway: 7.740060223559796 km


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
